In [6]:
import time # required to output the time at which the script was run
from sys import stdout # this import is need to flush python output to the stdout (instead of leaving it
# in the buffer
from sys import argv # this import is needed in order for the script to handle command line arguments
import socket # this import allows us to access the name of the computer that the script is being run on
import re
from os import system
import malariagen_data
import allel
import scipy.cluster
from scipy import stats
from scipy.stats import spearmanr
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import matplotlib.patches as mpatches
import multiprocessing

In [ ]:
sample_sets= ["1288-VO-UG-DONNELLY-VMF00168","1288-VO-UG-DONNELLY-VMF00219"]
sample_query= ("aim_species == 'gambiae'")



In [ ]:
#load malariagen data, metadata and edit

#####################################################################################################################################################
ag3 = malariagen_data.Ag3(pre = True)
# Initially, we don't use a sample query
meta = ag3.sample_metadata(
    sample_sets=sample_sets, 
)
# Remove decimal and numbers after it in the "partner_sample_id" column not present in our meta data
meta['partner_sample_id'] = meta['partner_sample_id'].str.split('.').str[0]
print(meta.shape)

# Convert "partner_sample_id" column in meta to float64 to match llineup_meta dtype
meta['partner_sample_id'] = pd.to_numeric(meta['partner_sample_id'])
#llineup trial metadata
llineup_meta = pd.read_csv('./llineup_publication/Data/ento_geno_plasmo_data.csv',
                       index_col = 0,
                      ).query("species=='An. gambiae'")
llineup_meta= llineup_meta.drop_duplicates(subset=['wgs.sample.id'])

wrong_row = (llineup_meta['wgs.sample.id'] == '107506144173') & (llineup_meta['Sample.identifier'] == '60')
llineup_meta.loc[wrong_row, 'wgs.sample.id'] = '107506144500'
llineup_meta.set_index('wgs.sample.id', inplace = True)
llineup_meta = llineup_meta.reindex(index=meta['partner_sample_id'])#match order in malariagen meta data
llineup_meta=llineup_meta.reset_index()
llineup_meta['sample_id'] =meta['sample_id'] #create sample id to merge with ag3 metadata
sample_query_filter = (meta['aim_species'] == 'gambiae') & (meta['sex_call'] == 'F')

llineup_meta['control_phase'] = llineup_meta['RND'].map({1: 'pre', 5: 'post'}).fillna('intermediate') #not needed but include if you want to plot control_phase

llineup_meta.rename(columns={'LLIN.actual':'llin_actual'}, inplace = True)
llineup_meta.dropna(subset=['Location'], inplace=True)
ag3.add_extra_metadata(llineup_meta)
# Re-download the metadata with the additional columns
meta = ag3.sample_metadata(
    sample_sets=sample_sets,
    sample_query=sample_query 
)

meta.dropna(subset=['Location'], inplace=True)
meta.set_index('sample_id', inplace=True)

#meta = meta[meta['RND'].isin([1,5]) & (meta['llin_actual']=='Non-PBO LLIN')]

(1222, 57)                           


In [38]:
meta

,partner_sample_id_x,contributor,country,location,year,month,latitude,longitude,sex_call,sample_set,...,survey,LLIN.intend,llin_actual,Net.intend,Net.actual,Wave,Location,subregions,species,control_phase
sample_id,,,,,,,,,,,,,,,,,,,,,
VBS50295-6645STDY11194062,109410010012,Martin Donnelly,Uganda,9410,2017,5,1.075,33.390,F,1288-VO-UG-DONNELLY-VMF00168,...,baseline,PBO LLIN,PBO LLIN,PermaNet-3,PermaNet-3,3.0,East,East Central,An. gambiae,pre
VBS50296-6645STDY11194063,109410010006,Martin Donnelly,Uganda,9410,2017,5,1.075,33.390,F,1288-VO-UG-DONNELLY-VMF00168,...,baseline,PBO LLIN,PBO LLIN,PermaNet-3,PermaNet-3,3.0,East,East Central,An. gambiae,pre
VBS50297-6645STDY11194064,109410010005,Martin Donnelly,Uganda,9410,2017,5,1.075,33.390,F,1288-VO-UG-DONNELLY-VMF00168,...,baseline,PBO LLIN,PBO LLIN,PermaNet-3,PermaNet-3,3.0,East,East Central,An. gambiae,pre
VBS50298-6645STDY11194065,109410010001,Martin Donnelly,Uganda,9410,2017,6,1.075,33.390,F,1288-VO-UG-DONNELLY-VMF00168,...,baseline,PBO LLIN,PBO LLIN,PermaNet-3,PermaNet-3,3.0,East,East Central,An. gambiae,pre
VBS50299-6645STDY11194066,109410010024,Martin Donnelly,Uganda,9410,2017,6,1.075,33.390,F,1288-VO-UG-DONNELLY-VMF00168,...,baseline,PBO LLIN,PBO LLIN,PermaNet-3,PermaNet-3,3.0,East,East Central,An. gambiae,pre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VBS65916-6645STDY13044822,506806007004.000,Martin Donnelly,Uganda,6806,2019,5,2.194,33.722,F,1288-VO-UG-DONNELLY-VMF00219,...,25 months,PBO LLIN,PBO LLIN,OlysetPlus,OlysetPlus,2.0,East,Mid-Eastern,An. gambiae,post
VBS65918-6645STDY13044824,506806007006.000,Martin Donnelly,Uganda,6806,2019,5,2.194,33.722,F,1288-VO-UG-DONNELLY-VMF00219,...,25 months,PBO LLIN,PBO LLIN,OlysetPlus,OlysetPlus,2.0,East,Mid-Eastern,An. gambiae,post
VBS65919-6645STDY13044825,506807032001.000,Martin Donnelly,Uganda,6807,2019,5,2.239,33.577,F,1288-VO-UG-DONNELLY-VMF00219,...,25 months,PBO LLIN,PBO LLIN,OlysetPlus,OlysetPlus,2.0,East,Mid-Eastern,An. gambiae,post


In [ ]:
# Identify tagging snps for duplicated genes

# Identify samples that have main dups # do this for all dup variants of that genes that you are interested in
discordant_read_calls_dup1a = (
    ag3
    .cnv_discordant_read_calls(contig="2R", 
                               sample_sets=sample_sets,
                               sample_query = sample_query
                               )
    .set_index(samples="sample_id", variants="variant_id")
    .sel(variants='Cyp6aap_Dup1a') #if you don't know the CNV variants, check by running this code without including
                                   #.sel then view by running discordant_read_calls.variants.value
    .sel(samples=meta.index.astype(str))
)
#discordant_read_calls_dup1a
Dup1_calls = discordant_read_calls_dup1a.call_genotype.values



# get a filter for excluding samples with poor quality HMM data
gene_copy_number = (
    ag3.gene_cnv(region="2R", 
                 sample_sets=sample_sets,
                 sample_query = sample_query,
                 max_coverage_variance = None
)
    .set_index(genes="gene_id", samples="sample_id")
    .sel(samples=meta.index.astype(str))
)

# Select data for genes of interest, excluding samples with poor quality HMM data
cyp6aa1_gene_copy_number = (
    gene_copy_number["CN_mode"]
    .sel(genes='AGAP002862')
    .transpose()
    .to_pandas()
)

good_variance = ~gene_copy_number.sample_is_high_variance.values


Dup1_copy_number = (cyp6aa1_gene_copy_number[good_variance]-2) * Dup1_calls[good_variance]
meta['dup1_copy_number'] = Dup1_copy_number.reindex(meta.index)

Load CNV HMM data:   0%|          | 0/176 [00:00<?, ?it/s]

Compute modal gene copy number:   0%|          | 0/3668 [00:00<?, ?it/s]

samples
VBS50275-6645STDY11194051    3
VBS50276-6645STDY11194052    2
VBS50283-6645STDY11194053    0
VBS50286-6645STDY11194055    2
VBS50287-6645STDY11194056    4
                            ..
VBS67021-6645STDY13057665    2
VBS67022-6645STDY13057666    1
VBS67024-6645STDY13057668    2
VBS67025-6645STDY13057669    2
VBS67026-6645STDY13057670    3
Length: 1005, dtype: int8

In [ ]:
#H3: dup1 copy number  change between round 1 and 5
meta=meta.dropna(subset=['dup1_copy_number'])
# Separate the dup1_copy_number values for each round
dup1_round1 = meta[meta['RND'] == 1]['dup1_copy_number']
dup1_round5 = meta[meta['RND'] == 5]['dup1_copy_number']

# Perform the Mann-Whitney U test
result = stats.mannwhitneyu(dup1_round1, dup1_round5)

# Print the result
print(result)

MannwhitneyuResult(statistic=55077.5, pvalue=0.3521640721682472)


In [ ]:
#H4 dup1 copy number  change between round 1 and 5 in nonpbo
meta_npbo= meta[(meta['llin_actual']=='Non-PBO LLIN')]
meta_npbo=meta_npbo.dropna(subset=['dup1_copy_number'])
# Separate the dup1_copy_number values for each round
dup1_round1 = meta_npbo[meta_npbo['RND'] == 1]['dup1_copy_number']
dup1_round5 = meta_npbo[meta_npbo['RND'] == 5]['dup1_copy_number']

# Perform the Mann-Whitney U test
result = stats.mannwhitneyu(dup1_round1, dup1_round5)

# Print the result
print(result)

MannwhitneyuResult(statistic=24634.0, pvalue=0.14629999825598808)


In [69]:
#h4 dup1 copy number  change between round 1 and 5 in pbo
meta_pbo= meta[(meta['llin_actual']=='PBO LLIN')]
meta_pbo=meta_pbo.dropna(subset=['dup1_copy_number'])
# Separate the dup1_copy_number values for each round
dup1_round1 = meta_pbo[meta_pbo['RND'] == 1]['dup1_copy_number']
dup1_round5 = meta_pbo[meta_pbo['RND'] == 5]['dup1_copy_number']

# Perform the Mann-Whitney U test
result = stats.mannwhitneyu(dup1_round1, dup1_round5)

# Print the result
print(result)

MannwhitneyuResult(statistic=4377.0, pvalue=0.370947240300443)


In [ ]:
# Identify tagging snps for duplicated genes

# Identify samples that have main dups # do this for all dup variants of that genes that you are interested in
discordant_read_calls_dup1a = (
    ag3
    .cnv_discordant_read_calls(contig="2R", 
                               sample_sets=sample_sets,
                               sample_query = sample_query
                               )
    .set_index(samples="sample_id", variants="variant_id")
    .sel(variants='Cyp6aap_Dup1a') #if you don't know the CNV variants, check by running this code without including
                                   #.sel then view by running discordant_read_calls.variants.value
    .sel(samples=meta.index.astype(str))
)
#discordant_read_calls_dup1a
Dup1_calls = discordant_read_calls_dup1a.call_genotype.values



# get a filter for excluding samples with poor quality HMM data
gene_copy_number = (
    ag3.gene_cnv(region="2R", 
                 sample_sets=sample_sets,
                 sample_query = sample_query,
                 max_coverage_variance = None
)
    .set_index(genes="gene_id", samples="sample_id")
    .sel(samples=meta.index.astype(str))
)

# Select data for genes of interest, excluding samples with poor quality HMM data
cyp6aa1_gene_copy_number = (
    gene_copy_number["CN_mode"]
    .sel(genes='AGAP002862')
    .transpose()
    .to_pandas()
)

good_variance = ~gene_copy_number.sample_is_high_variance.values


Dup1_copy_number = (cyp6aa1_gene_copy_number[good_variance]-2) * Dup1_calls[good_variance]

#now we get the snp tag for dup1

#haplotype calls withing the region where the dup is
region='2R:28470000-28500000'

haps = (
    ag3.haplotypes(region = region, 
                   analysis = 'gamb_colu', 
                   sample_sets = sample_sets,
                   sample_query = sample_query
                   )
    .set_index(samples = 'sample_id')
    .sel(samples=meta.index.astype(str))
)

haps_Dup = haps.isel(samples = good_variance)

# Create haplotype array
hap_array_Dup = allel.GenotypeArray(haps_Dup.call_genotype).to_haplotypes()

# Get allele counts
ac = hap_array_Dup.count_alleles()

# Identify segregating non-singletons. 
non_singleton = ac.min(1) > 1

# Filter the datasets, removing non-segregating and singleton variants 
haps_Dup = haps_Dup.isel(variants = np.where(non_singleton)[0])
genotypes = haps_Dup.call_genotype.values.sum(2)


SNP_cor_dup1 = np.apply_along_axis(lambda x: spearmanr((x>0).astype('int'), 
                                                  Dup1_calls[good_variance]), 
                                    1, 
                                    genotypes)[:, 0]
SNP_cor_dup1 = np.abs(SNP_cor_dup1)

Dup1_proxy_SNP_position = haps_Dup.variant_position[np.nanargmax(SNP_cor_dup1)].values
Dup1_proxy = genotypes[np.nanargmax(SNP_cor_dup1)]


print(f'The Dup1 proxy SNP is at positions {Dup1_proxy_SNP_position}')

print(f'The maximum correlation coefficient with the Dup is {np.nanmax(SNP_cor_dup1)}')





Load CNV HMM data:   0%|          | 0/176 [00:00<?, ?it/s]

Compute modal gene copy number:   0%|          | 0/3668 [00:00<?, ?it/s]

/tmp/ipykernel_1328677/2610785262.py:64: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  SNP_cor_dup1 = np.apply_along_axis(lambda x: spearmanr((x>0).astype('int'),


The Dup1 proxy SNP is at positions 28470419
The maximum correlation coefficient with the Dup is 0.5732864158771411


In [12]:

#Specify the colours and labels  of information  to add on the dendongram


#Location and other rows you want to visualise
location = np.unique(meta.Location)
num_location = len(location)
location_palette = sns.color_palette('viridis', num_location)
location_colours = {location[i]: location_palette[i] for i in range(num_location)}

#control_phase_colours = {'pre': 'olive', 'post': 'khaki', 'intermediate':'#BDB76B'}



# Now add extra rows for the SNPs and dup proxy

#dup tagging snps
dup1_proxy =28470419
dup5_proxy =28471934

#cyp6p region snps 

region = '2R:28463444-28499726'
array_of_all_snps =np.array([28470419,28471934,28492268]) # include all snps you want to plot and also snps tagging the dups
array_of_all_snps_labels = ['Cyp6aa1_dup1', 'Cyp6aa1_dup','Cyp6p3_L292V'] # name of the snps if you want to specify, if not include the snps pos as above but with ''



#plot the haplotypes in the region of interest as determined by H12 peak modelling

haps = (
    ag3.haplotypes(region = region, 
                   analysis = 'gamb_colu', 
                   sample_sets = sample_sets,
                   sample_query = sample_query)
    .set_index(samples = 'sample_id')
    .sel(samples = meta.index.astype(str))

)
hapsamples = np.array(np.repeat(haps['samples'], 2))

location_labels = meta.loc[hapsamples, 'Location'].values
#control_phase_labels = meta.set_index('sample_id').loc[hapsamples, 'control_phase'].values


haparray = allel.GenotypeArray(haps['call_genotype']).to_haplotypes()
hapnames = np.concatenate([[x + 'a', x + 'b'] for x in haps['samples'].values])

dist = allel.pairwise_distance(haparray, metric = 'hamming')

site_filter = ag3.snp_calls(region=region, sample_sets=sample_sets)['variant_filter_pass_gamb_colu']
n_bases = np.sum(site_filter.values)
dist_dxy = dist * haparray.n_variants / n_bases


min_cluster_size = 20
# A function to identify haplotyde clusters from a distance matrix.
def find_clusters(dist, n, threshold=0.001, method='complete'):
        # build hierarchy
        clust = scipy.cluster.hierarchy.linkage(dist, method=method)
        # find clusters
        f = scipy.cluster.hierarchy.fcluster(clust, threshold,
                                             criterion='distance')
        # compute cluster sizes
        fsz = np.bincount(f)
        # sort largest first
        fsort = np.argsort(fsz)[::-1]
        # take largest n
        fsort = fsort[:n]
        # get haplotype indices for each cluster
        clusters = [set(np.nonzero(f == i)[0]) for i in fsort]
        return clusters

focal_clusters = find_clusters(dist_dxy, n=20, threshold=0.001)
large_focal_clusters = [cluster for cluster in focal_clusters if len(cluster) > min_cluster_size]
focal_cluster1_haps = hapnames[list(large_focal_clusters[0])]


# Function to assign sample haplotypes to clusters
def assign_sample_haplotypes_parallel(cluster_ids):
    cluster_samples = np.array(hapsamples[list(cluster_ids)]).flatten()
    sample_ids = meta.index.astype(str).values
    sample_hap_genotype = np.array([np.sum(np.isin(cluster_samples, sample)) for sample in sample_ids])
    return sample_hap_genotype

# Define number of processes
num_processes = min(20, multiprocessing.cpu_count())  # Ensures it doesn’t exceed available cores

# Run in parallel
with multiprocessing.Pool(processes=num_processes) as pool:
    cluster_assignments = pool.map(assign_sample_haplotypes_parallel, large_focal_clusters)

# Store results in meta DataFrame
for i, assignment in enumerate(cluster_assignments):
    meta[f'cluster_{i+1}'] = assignment

# Prepare the output table
output_columns = ['Location', 'llin_actual', 'control_phase', 'RND', 'HSD'] + [f'cluster_{i+1}' for i in range(len(large_focal_clusters))]
output_table = meta[output_columns]

# Save to CSV
output_filename = f'cluster_assignments_{region}.csv'
output_table.to_csv(output_filename, sep='\t', index_label='sample_name')


z1 = scipy.cluster.hierarchy.linkage(dist_dxy, method="complete")
r1 = scipy.cluster.hierarchy.dendrogram(
        z1, no_labels=True, count_sort=True,
        color_threshold=0, no_plot=True,
        above_threshold_color='k')

# Function to find the span of each cluster

def truspan(cluster, r):
	# get the index of the cluster haps in the dendrogram list of all haps
	cluster_leaves = sorted([r['leaves'].index(i) for i in cluster])
	# are these indices monotonic - they should be!
	x = np.asarray(cluster_leaves)
	dx = np.diff(x)
	mon = np.all(dx == 1)
	assert mon
	return min(cluster_leaves), max(cluster_leaves)

v_span = [truspan(cluster, r1) for cluster in large_focal_clusters]
cluster_lab = ['C' + str(i+1) for i in range(len(large_focal_clusters))]


# add snps to the plot

 #full_region = f'2R:{np.min(array_of_all_snps)}-{np.max(array_of_all_snps)}' #change chrom
full_region = '2L:2300000-2900000'

full_region_haps = (ag3.haplotypes(
    region = region,
    analysis = 'gamb_colu',
    sample_sets = sample_sets,
    sample_query = sample_query,)
    .set_index(samples = 'sample_id')
    .sel(samples = meta.index.astype(str)))

indexed_haps = full_region_haps.set_index(variants = 'variant_position')
snp_calls = allel.GenotypeArray(indexed_haps.sel(variants = array_of_all_snps).call_genotype).to_haplotypes()
#colour schemes
snp_colour_scheme = {0: 'lightgrey', 1: 'black'}

dup1_colour_scheme = {0: 'lightgrey', 1: 'blue'}
dup5_colour_scheme = {0: 'lightgrey', 1: 'red'}

#these_labels = [location_labels, control_phase_labels]
these_labels = [location_labels]#, kdr_haps_labels]
#these_colour_schemes = [location_colours, control_phase_colours]
these_colour_schemes = [location_colours]#, kdr_haps_colours]
#these_label_names = ['location', 'control phase']
these_label_names = ['location']#, 'KDR haps']
for snp, snp_label in zip(array_of_all_snps, array_of_all_snps_labels):
     #if no dups then comment out
    if snp == dup1_proxy:
        these_labels.append(snp_calls[list(array_of_all_snps).index(snp)])
        these_colour_schemes.append(dup1_colour_scheme)  # Apply different color scheme 
        these_label_names.append('Cyp6aa1_dup1')  # Custom label instead of SNP position
          #if no dups then comment out
    elif snp == dup5_proxy:
        these_labels.append(snp_calls[list(array_of_all_snps).index(snp)])
        these_colour_schemes.append(dup5_colour_scheme)  # Apply different color scheme
        these_label_names.append('Cyp6aa1_dup5')  # Custom label instead of SNP position
          #if no snps then comment out
    else:
        these_labels.append(snp_calls[list(array_of_all_snps).index(snp)])
        these_colour_schemes.append(snp_colour_scheme)
        these_label_names.append(snp_label)
        #these_label_names.append(array_of_all_snps_labels)   # Keep original SNP labels


In [14]:

def draw_hap_cluster_plot2(z, h, 
                          colour_schemes, labels, label_names,
                          cluster_labels, vspans,
                          add_legend=True, title='',
                          fn=None):
    # Adjust GridSpec to include the cluster brackets
    gs = GridSpec(len(colour_schemes) + 2, 1, height_ratios=[6.0] + [0.4] * len(colour_schemes) + [0.8])
    fig = plt.figure(figsize=(15, (7 + 0.58 * len(colour_schemes) + 1)))
    plt.subplots_adjust(hspace=0.02)
    ax1 = plt.subplot(gs[0])
    sns.despine(ax=ax1, offset=5, bottom=True, top=True)
    r = scipy.cluster.hierarchy.dendrogram(z, no_labels=True, count_sort=True,
                                           color_threshold=0,
                                           above_threshold_color='k',
                                           ax=ax1)
    ax1.spines['top'].set_visible(False)
    ax1.set_xticks([])
    ax1.set_ylim(0)
    
    for i in range(len(colour_schemes)):
        this_ax = fig.add_subplot(gs[i + 1])
        l = np.array([labels[i][hap] for hap in r['leaves']])
        
        # Color SNP haplotypes dynamically
        hap_clrs = [colour_schemes[i].get(p, 'lightgrey') for p in l]
        
        this_ax.broken_barh(xranges=[(j, 1) for j in range(h.shape[1])],
                            yrange=(0, 1),
                            color=hap_clrs)
        sns.despine(ax=this_ax, offset=5, left=True, bottom=True, top=True)
        this_ax.set_xticks([])
        this_ax.set_yticks([])
        this_ax.set_xlim(0, h.shape[1])
        this_ax.yaxis.set_label_position('left')
        this_ax.set_ylabel(label_names[i], rotation=0, ha='right', va='center')
    
    # Add a cleaner legend dynamically
    if add_legend:
        legend_patches = []

        # Add categorical variables first
        for val in np.unique(labels[0]):  # Location
            if val in colour_schemes[0]:
                legend_patches.append(mpatches.Patch(color=colour_schemes[0][val], label=f'Location: {val}'))
        
        # Add SNPs as a single category
        legend_patches.append(mpatches.Patch(color='black', label='SNP: Alt'))
        legend_patches.append(mpatches.Patch(color='lightgrey', label='SNP: Ref'))
        
        # Explicitly add `dup1` and `dup5`  # comment out if no dups
        legend_patches.append(mpatches.Patch(color='blue', label='Dup1'))
        legend_patches.append(mpatches.Patch(color='red', label='Dup5'))
        legend_patches.append(mpatches.Patch(color='lightgrey', label='No_dup'))
        ax1.legend(handles=legend_patches, title="", loc='right', frameon=False)
    
    # Cluster brackets subplot
    ax_clu = fig.add_subplot(gs[-1])
    sns.despine(ax=ax_clu, bottom=True, left=True)
    ax_clu.set_xlim(0, h.shape[1])
    ax_clu.set_ylim(0, 1)
    
    for lbl, (xmin, xmax) in zip(cluster_labels, vspans):
        if lbl:
            fraction = -20 / (xmax - xmin)
            ax_clu.annotate("", ha='left', va='center',
                            xy=(xmin, 1), xycoords='data',
                            xytext=(xmax, 1), textcoords='data',
                            arrowprops=dict(arrowstyle="-",
                                            connectionstyle="bar,fraction=%.4f" % fraction))
            ax_clu.text((xmax + xmin) / 2, 0.1, lbl, va='top', ha='center')
    
    ax_clu.set_xticks([])
    ax_clu.set_yticks([])
    ax1.set_title(f'{title}')
    
    fig.patch.set_alpha(1)
    
    if fn:
        ext = re.sub('.*\\.', '', fn)
        plt.savefig(fn, format=ext)
    
    plt.show()


In [ ]:
# Plot region 1 haplotypes while viewing the SNPs of region2
draw_hap_cluster_plot2(z1, haparray, labels=these_labels,
                       cluster_labels=cluster_lab,
                      vspans=v_span,
                      colour_schemes = these_colour_schemes,
                      label_names = these_label_names,
					  fn = 'haplotype_cluster_test.png'
)
plt.show()